- last week we learnt the rolling model
- this week we would learn ensemble methods and intraday trading

# Path

In [1]:
HEAD_PATH = './'
DATA_PATH = HEAD_PATH + "data/stocks/"
SAVE_PATH = HEAD_PATH + "ckpt/"    # most of signal timeseries is here
TEMP_PATH = SAVE_PATH + "tmp pkl/"
OUTPUT_PATH = HEAD_PATH + 'output'

# signal result atr: signal evaluate results

In [2]:
from stock_helper import *
from stock_stats import *
from product_info import *
from imp import reload  
import stock_helper
import stock_stats
reload(stock_helper)
reload(stock_stats)

<module 'stock_stats' from '/mnt/d/A-Share/tmp/stock_jupyter/stock_stats.py'>

In [3]:
import warnings
 
warnings.filterwarnings('ignore')

import dask
from dask import compute, delayed
import sklearn.preprocessing

In [4]:
# CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
CORE_NUM = 10
## 16

In [5]:
product_list = ["600519", "000858", "000568", "600809", "002304"]

In [6]:
period = 4096

# Functions

In [7]:
## parallel generate the distribution of a signal
def par_get_all_signal(signal_name, file_list, product, period, SAVE_PATH="e:/intern"):
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file) ## signal
#         good = load(SAVE_PATH+"/good pkl/"+product+"/"+file) ## good singal
#         signal = S[good]
        signal = S
        moving_average(signal,period)
        chosen = (np.arange(len(signal))+1) % period==0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, SAVE_PATH+"/all signal/"+product+"."+signal_name+".pkl")
    
    
def evaluate_signal(signal, all_dates, product, min_pnl, min_num, 
                    CORE_NUM, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern", period=4096, split_str="2018", 
                    atr_filter=0, save_path="signal result",reverse=0):
    signal_name = signal+"."+str(period) ## signal name, with period
    all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
    open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
    thre_list = []
    for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
        thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
    thre_list = np.array(thre_list)
    thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
    
    if reverse>=0: ## trending signal
        print("reverse=1")
        trend_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=1, 
                                    atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse<=0: ## reversal signal
        print("reverse=-1")
        reverse_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=-1,
                                            atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse==0: ## both trending and reversal
        stat_result = OrderedDict([("trend.signal.stat", trend_signal_stat), ("reverse.signal.stat", reverse_signal_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".pkl")
    elif reverse==1: ## just trend
        save(trend_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".trend.pkl")
    elif reverse==-1: ## just reversal
        save(reverse_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".reverse.pkl")
        

# calc pnl given a signal
def get_signal_pnl(file, product, signal_name, thre_mat, reverse=1,  buy_tranct=1.5e-4, sell_tranct=11.5e-4,
                   max_spread=0.011,
                   HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH, atr_filter=0):
    ## load data
    data = load(DATA_PATH+product+"/"+file)
    S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file)
    pred = S*reverse
    atr = load(SAVE_PATH+"/tmp pkl/"+product+"/"+"atr.4096"+"/"+file)
    #n_bar = len(data)
    
    ## load signal
    
    ## we don't know the signal is positive correlated or negative correlated  
    #n_thre = len(thre_mat)
    date = np.array([x[0:10] for x in data["date.time"]])
    next_date = np.append(date[1:],'1')
    end_day = date!=next_date
    count = 0;
    n_day = sum(end_day)
    n_thre = np.shape(thre_mat)[0]
    all_pnl = np.zeros((n_day, n_thre))
    result = pd.DataFrame(data=OrderedDict([("open", thre_mat["open"].values), ("close", thre_mat["close"].values),
                               ("num", 0), ("avg.ret", 0), ("ret", 0)]), 
                          index=thre_mat.index)

    cur_spread = data["ask1"]-data["bid1"]
    for thre in thre_mat.iterrows():
        count = count+1
        buy = pred>thre[1]["open"]
        sell = pred<-thre[1]["open"]
        signal = pd.Series(data=0, index=data.index)
        position = signal.copy()
        signal[buy] = 1
        signal[sell] = -1
        signal[atr<atr_filter]=0
        scratch = -thre[1]["close"]
        position_pos = pd.Series(data=np.nan, index=data.index)
        position_pos.iloc[0] = 0
        position_pos[(signal==1) & (data["next.ask"]>0) & (data["next.bid"]>0) & (cur_spread<max_spread)] = 1
        position_pos[(pred< -scratch) & (data["next.bid"]>0) & (cur_spread<max_spread)] = 0
        position_pos.ffill(inplace=True)
        pre_pos = position_pos.shift(1)
        notional_position_pos = pd.Series(data=0, index=data.index)
        notional_position_pos[position_pos==1] = 1
        notional_position_pos[(position_pos==1) & (pre_pos==1)] = np.nan
        notional_position_pos[(notional_position_pos==1)] = 1/data["next.ask"][(notional_position_pos==1)]
        notional_position_pos.ffill(inplace=True)
        position_neg = pd.Series(data=np.nan, index=data.index)
        position_neg.iloc[0] = 0
        position_neg[(signal==-1) & (data["next.ask"]>0) & (data["next.bid"]>0) & (cur_spread<max_spread)] = -1
        position_neg[(pred> scratch) & (data["next.ask"]>0) & (cur_spread<max_spread)] = 0
        position_neg.ffill(inplace=True)
        pre_neg = position_neg.shift(1)
        notional_position_neg = pd.Series(data=0, index=data.index)
        notional_position_neg[position_neg==-1] = -1
        notional_position_neg[(position_neg==-1) & (pre_neg==-1)] = np.nan
        notional_position_neg[(notional_position_neg==-1)] = -1/data["next.bid"][(notional_position_neg==-1)]
        notional_position_neg.ffill(inplace=True)
        position = position_pos + position_neg
        notional_position = notional_position_pos+notional_position_neg
        #position[n_bar-1] = 0
        position.iloc[0] = 0
        position.iloc[-2:] = 0
        notional_position.iloc[0] = 0
        notional_position.iloc[-2:] = 0
        #change_pos = position - position.shift(1)
        #notional_change_pos = notional_position-notional_position.shift(1)
        change_pos = notional_position-notional_position.shift(1)
        change_pos.iloc[0] = 0
        #notional_change_pos.iloc[0] = 0
        change_base = pd.Series(data=0, index=data.index)
        change_buy = change_pos>0
        change_sell = change_pos<0
        change_base[change_buy] = data["next.ask"][change_buy]*(1+buy_tranct)*data["adjust"]
        change_base[change_sell] = data["next.bid"][change_sell]*(1-sell_tranct)*data["adjust"]
        raw_pnl = -(change_base*change_pos).cumsum()+notional_position*data["wpr"]
        final_pnl = -sum(change_base*change_pos) ## total pnl, there is a negative sign, because selling get money and buying pay money
        turnover = sum(change_base*abs(change_pos))
        num = sum((position!=0) & (change_pos!=0)) ## number of trades
        hld_period = sum(position!=0)   ## holding period
        daily_pnl = raw_pnl[end_day].reset_index(drop=True)
        pnl = np.append(daily_pnl[0], np.diff(daily_pnl))
        all_pnl[:,thre[0]] = pnl
        if (num==0):
            result.loc[thre[0], ("num","avg.ret","ret")] = (0,0,0)
        else:
            result.loc[thre[0],("num", "avg.ret", "ret", )] = (num, final_pnl/num, final_pnl)
    return OrderedDict([("all.pnl", all_pnl), ("result", result), ("date", date[end_day])])

    
from collections import OrderedDict
def get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=1, 
                     atr_filter=0, HEAD_PATH="d:/intern", SAVE_PATH="d:/intern"):
    train_sample = all_dates<split_str ## training samples
    test_sample = all_dates>split_str ## testing samples
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=1,
                                HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH,atr_filter=atr_filter)
        train_result = compute([delayed(f_par)(file) for file in all_dates[train_sample]])[0] ## get training result
    train_stat = get_hft_summary(train_result, thre_mat) ## get training result statistics
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=1,
                                HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH,atr_filter=atr_filter)
        test_result = compute([delayed(f_par)(file) for file in all_dates[test_sample]])[0] ## get testing result
    test_stat = get_hft_summary(test_result, thre_mat) ## get testing result statistics
    return OrderedDict([("train.stat", train_stat), ("test.stat", test_stat)])

    
def evaluate_signal(signal, all_dates, product, min_pnl, min_num, 
                    CORE_NUM, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern", period=4096, split_str="2018", 
                    atr_filter=0, save_path="signal result",reverse=0):
    signal_name = signal+"."+str(period) ## signal name, with period
    all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
    open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
    thre_list = []
    for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
        thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
    thre_list = np.array(thre_list)
    thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
    
    if reverse>=0: ## trending signal
        print("reverse=1")
        trend_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=1, 
                                    atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse<=0: ## reversal signal
        print("reverse=-1")
        reverse_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=-1,
                                            atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse==0: ## both trending and reversal
        stat_result = OrderedDict([("trend.signal.stat", trend_signal_stat), ("reverse.signal.stat", reverse_signal_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".pkl")
    elif reverse==1: ## just trend
        save(trend_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".trend.pkl")
    elif reverse==-1: ## just reversal
        save(reverse_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".reverse.pkl")
        
# summarize pnl over thresholds
def get_hft_summary(result, thre_mat):
    n_thre = np.shape(thre_mat)[0]
    all_pnl = np.zeros((0,n_thre))
    all_dates = np.array([])
    for i in range(len(result)):
        all_pnl =  np.concatenate((all_pnl,result[i]["all.pnl"]),axis=0)
        all_dates = np.append(all_dates, result[i]["date"])
    stat = result[0]["result"].iloc[:,2:]
    for i in range(1,len(result)):
        stat = stat+result[i]["result"].iloc[:,2:]
    stat["avg.ret"]=stat["ret"]/stat["num"]
    
    total_ret = all_pnl.sum(0)
    total_sharpe = np.zeros(n_thre)
    total_drawdown = np.zeros(n_thre)
    total_max_drawdown = np.zeros(n_thre)
    for i in range(n_thre):
        total_sharpe[i] = sharpe(all_pnl[:,i])
        total_drawdown[i] = drawdown(all_pnl[:,i])
        total_max_drawdown[i] = max_drawdown(all_pnl[:,i])
    final_result = pd.DataFrame(data=OrderedDict([("open", thre_mat["open"]), ("close", thre_mat["close"]), ("num", stat["num"]),
                                                  ("avg.ret", stat["avg.ret"]), ("total.ret",total_ret), ("sharpe", total_sharpe),
                                                  ("drawdown", total_drawdown), ("max.drawdown", total_max_drawdown),
                                                 ("mar", total_ret/total_max_drawdown)]), 
                                index=thre_mat.index)
    return OrderedDict([("final.result", final_result), ("daily.pnl", all_pnl), ("date", all_dates)])

In [8]:
# ## coutn the number of signals of every day
# def count_daily_num(x, period):
#     data = load(x)
#     return np.floor(np.shape(data)[0]/period)

## get multiple samples of the singal matrix
def get_multiple_sample_signal(good_night_files, sample, product, signal_name, period, daily_num, mod=np.array([0])):
    n_samples = int(sum(daily_num[sample])) ## number of samples
    n_mod = len(mod) ## number of different copies of the same signal
    if n_mod==1: ## if there is one copy then the same as before
        all_signal = np.zeros(n_samples)
    else:
        all_signal = np.zeros((n_mod, n_samples)) ## if there are multiple copies then use a matrix for one signal instead
    cur = 0
    for file in good_night_files[sample]: ## iterate for files
        S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file)
        for i in range(n_mod):
            signal = S[(np.arange(len(S))+1) % period == mod[i]]
            ## the ret.cor has some bad records
            signal[np.isnan(signal)] = 0
            signal[np.isinf(signal)] = 0
            if mod[i]==0: 
                cur_end = np.min((cur+len(signal),n_samples)) ## to avoid exceed the number of samoes, mod[i]==0 has most signals, 
                ## so if it doesn't exceed, others would not exceed
            if n_mod==1:
                all_signal[cur:cur_end] = signal[0:(cur_end-cur)]
            else:
                all_signal[i, cur:cur_end] = signal[0:(cur_end-cur)]
        cur = cur_end
    return all_signal

In [9]:
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)


In [10]:
# SAVE_PATH = "e:/intern"
all_dates_x = os.listdir(DATA_PATH + product_list[0])
all_dates_y = os.listdir(DATA_PATH + product_list[1])
all_dates = np.array(list(set(all_dates_x) & set(all_dates_y)))
all_dates.sort()
train_sample = all_dates<"2017"
test_sample = all_dates>"2017"


In [11]:
os.makedirs(SAVE_PATH+"/all signal", exist_ok=True)
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["", "range", "std", "trend.index"]
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
signal_list = [signal+".4096" for signal in all_signal_list]
np.append(signal_list, ["ret.4096", "ret.4096.001", "ret.4096.002"])
n_signal = len(signal_list)

In [12]:
n_signal

28

In [13]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import lasso_path, LassoCV,RidgeCV

from sklearn.preprocessing import StandardScaler
#y_signal = "ret."+str(period)+".004"
y_signal = "ret."+str(period)+".002"

In [14]:
mod = np.arange(0, np.floor(period/10)*10, np.floor(period/10))
n_mod = len(mod)
mod
## array([   0.,  409.,  818., 1227., 1636., 2045., 2454., 2863., 3272.,
##        3681.])

array([   0.,  409.,  818., 1227., 1636., 2045., 2454., 2863., 3272.,
       3681.])

In [15]:
n_mod

10

In [16]:
# SAVE_PATH = "e:/intern"
os.makedirs(SAVE_PATH+"/train test array", exist_ok=True)

# 1st Ensemble

## All sets of signals

In [17]:
%%time
for product in product_list:
    print(product)
    daily_num = load(SAVE_PATH+"/daily num/"+product+".pkl")
    n_train = sum(daily_num[train_sample])
    n_test = sum(daily_num[test_sample])
    train_array = np.zeros((n_mod, int(n_train), n_signal+1))
    for i in range(n_signal):
        train_array[:,:,i] = get_multiple_sample_signal(all_dates, train_sample, product, signal_list[i], period, daily_num, mod)
    train_array[:,:,n_signal] = get_multiple_sample_signal(all_dates, train_sample, product, "ret."+str(period), period, daily_num, mod)
    save(train_array, SAVE_PATH+"/train test array/"+product+".train.array.pkl")
    
# 600519
# 000858
# 000568
# 600809
# 002304
# Wall time: 2min 53s



600519
000858
000568
600809
002304
CPU times: user 1min 17s, sys: 8.13 s, total: 1min 25s
Wall time: 4min 56s


In [18]:
all_train_array = dict([])
for product in product_list:
    all_train_array[product] = load(SAVE_PATH+"/train test array/"+product+".train.array.pkl")

## Coefficients of all models

In [19]:
## get the result of 10 models
## save them in to a model_coef dictionary
nfold = 10
model_coef = dict([])
for product in product_list:
    model_coef[product] = np.zeros(n_signal)
for i_mod in range(n_mod):
    train_std_mat = dict([])
    x_train = np.zeros((0, n_signal))
    y_train = np.array([])
    for product in product_list:
        cur_mat = all_train_array[product][i_mod,:,:]
        train_std_mat[product] = np.std(cur_mat, axis=0)
        x_train = np.vstack((x_train, cur_mat[:,:n_signal]/train_std_mat[product][:n_signal]))
        y_train = np.concatenate((y_train, cur_mat[:,n_signal]/train_std_mat[product][n_signal]))
    scaler =  StandardScaler(copy=True, with_mean=False, with_std=True)
    scaler.fit(x_train)
    x_std = np.sqrt(scaler.var_)
    x_train_normal = scaler.transform(x_train)
    model = LassoCV(n_alphas=100, fit_intercept=False, cv=10, max_iter=10000).fit(x_train_normal, y_train)
    fit_coef = model.coef_/x_std
    for product in product_list:
        coef = fit_coef*train_std_mat[product][n_signal]/train_std_mat[product][:n_signal]
        model_coef[product] = model_coef[product]+coef/n_mod


In [20]:
save(model_coef, SAVE_PATH+"/wine.ensemble.model.pkl")

In [21]:
model_coef = load(SAVE_PATH+"/wine.ensemble.model.pkl")

In [22]:
model_coef

# {'600519': array([ 6.56638999e-02,  8.67937178e-03,  1.12820595e-02, -3.45893863e-04,
#         -1.69496053e-01,  8.69035647e-04, -6.94687572e-04, -7.55077389e-03,
#         -2.68724390e-03, -1.85624470e-03, -3.88807539e-04,  9.17066951e-03,
#          2.31254444e-04, -2.70780985e-06, -1.40674802e-04, -1.03465109e-03,
#         -4.04187711e-04,  5.72359513e-04,  1.52929013e-02,  8.89410834e-05,
#          0.00000000e+00,  9.71755727e-02,  9.47852799e-03,  1.34725785e-03,
#          3.03069677e-03, -1.38021409e-01, -5.03610604e-05, -6.69183987e-04]),
#  '000858': array([ 5.79486801e-02,  6.10839453e-03,  1.10598584e-02, -3.22407674e-04,
#         -1.69352624e-01,  8.31988703e-04, -6.82160049e-04, -6.52527383e-02,
#         -2.26125063e-02, -1.75662348e-02, -3.43397293e-03,  7.12125531e-02,
#          2.14439807e-03, -6.11855953e-05, -1.93167988e-03, -8.52036979e-03,
#         -3.75934734e-03,  4.94148310e-03,  1.37450318e-01,  8.45934873e-04,
#          0.00000000e+00,  8.62163041e-02,  6.29179737e-03,  1.28061939e-03,
#          2.96175100e-03, -1.42302079e-01, -5.25622524e-05, -6.52758840e-04]),
#  '000568': array([ 8.32584057e-02,  9.84656518e-03,  1.46337913e-02, -4.23444174e-04,
#         -1.71655002e-01,  1.08816317e-03, -9.15574785e-04, -6.47137947e-02,
#         -2.65663540e-02, -1.63574481e-02, -3.30726612e-03,  6.26513661e-02,
#          1.99876226e-03, -4.23711339e-05, -1.46926791e-03, -9.92514438e-03,
#         -3.69774491e-03,  4.84810238e-03,  1.00029357e-01,  8.03378756e-04,
#          0.00000000e+00,  1.23994380e-01,  1.01834584e-02,  1.72775025e-03,
#          3.89680821e-03, -1.39928549e-01, -7.02822910e-05, -8.66623287e-04]),
#  '600809': array([ 1.07634511e-01,  1.49424196e-02,  1.94785845e-02, -5.76943838e-04,
#         -1.63945984e-01,  1.49789162e-03, -1.23145725e-03, -3.61749942e-02,
#         -1.47730775e-02, -9.35304243e-03, -1.87144176e-03,  2.64820577e-02,
#          1.16682101e-03,  3.23497973e-06, -7.42753736e-04, -5.31543897e-03,
#         -1.95948488e-03,  2.69090959e-03,  4.66702819e-02,  4.47707009e-04,
#          0.00000000e+00,  1.61912945e-01,  1.49810619e-02,  2.26479427e-03,
#          5.14113786e-03, -1.31605803e-01, -8.56769785e-05, -1.14695631e-03]),
#  '002304': array([ 1.03240934e-01,  1.65143820e-02,  1.82405970e-02, -4.99656748e-04,
#         -1.65016813e-01,  1.42759195e-03, -1.15023710e-03, -4.06148223e-03,
#         -5.65340222e-03, -2.37631008e-03, -5.01234678e-04,  4.52183688e-03,
#          3.15272028e-04,  4.74789451e-06,  2.48310890e-04, -1.47088241e-03,
#         -4.42081519e-04,  8.68033420e-04,  3.87059949e-03,  1.00663737e-04,
#          0.00000000e+00,  1.51970946e-01,  1.79941332e-02,  2.11798803e-03,
#          4.70395460e-03, -1.39872776e-01, -8.25842740e-05, -1.09728937e-03])}

{'600519': array([ 6.56638999e-02,  8.67937178e-03,  1.12820595e-02, -3.45893863e-04,
        -1.69496053e-01,  8.69035647e-04, -6.94687572e-04, -7.55077389e-03,
        -2.68724390e-03, -1.85624470e-03, -3.88807539e-04,  9.17066951e-03,
         2.31254444e-04, -2.70780985e-06, -1.40674802e-04, -1.03465109e-03,
        -4.04187711e-04,  5.72359513e-04,  1.52929013e-02,  8.89410834e-05,
         0.00000000e+00,  9.71755727e-02,  9.47852799e-03,  1.34725785e-03,
         3.03069677e-03, -1.38021409e-01, -5.03610604e-05, -6.69183987e-04]),
 '000858': array([ 5.79486801e-02,  6.10839453e-03,  1.10598584e-02, -3.22407674e-04,
        -1.69352624e-01,  8.31988703e-04, -6.82160049e-04, -6.52527383e-02,
        -2.26125063e-02, -1.75662348e-02, -3.43397293e-03,  7.12125531e-02,
         2.14439807e-03, -6.11855953e-05, -1.93167988e-03, -8.52036979e-03,
        -3.75934734e-03,  4.94148310e-03,  1.37450318e-01,  8.45934873e-04,
         0.00000000e+00,  8.62163041e-02,  6.29179737e-03,  1.2806

In [23]:
strat = "wine.ensemble.lasso."+str(period)
for product in product_list:
    os.makedirs(SAVE_PATH+"/tmp pkl/"+product+"/"+strat, exist_ok=True)

In [24]:
%%time
for product in product_list:
    print(product)
    coef = model_coef[product]
    parLapply(CORE_NUM, all_dates, par_get_daily_pred, 
              product=product, coef=coef, strat=strat, HEAD_PATH=SAVE_PATH, SAVE_PATH=SAVE_PATH)

# 600519
# 000858
# 000568
# 600809
# 002304
# Wall time: 6min 11s





600519
000858
000568
600809
002304
CPU times: user 430 ms, sys: 284 ms, total: 715 ms
Wall time: 2min 52s


In [25]:
%%time
for product in product_list:
    par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH=SAVE_PATH)

## Wall time: 8.99 s



CPU times: user 3.91 s, sys: 562 ms, total: 4.47 s
Wall time: 18.1 s


## Evaluate

In [26]:
%%time
for product in product_list:
    evaluate_signal("wine.ensemble.lasso", all_dates, product, 0.001, 20, CORE_NUM, SAVE_PATH, SAVE_PATH,
                        period=4096, split_str="2017",atr_filter=0.02, save_path="signal result atr", reverse=1)

    
# reverse=1
# reverse=1
# reverse=1
# reverse=1
# reverse=1
# Wall time: 28min 52s





reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
CPU times: user 536 ms, sys: 459 ms, total: 995 ms
Wall time: 9min 5s


In [27]:
for product in product_list:
    signal_stat = load(SAVE_PATH+"/signal result atr/"+product+"."+"wine.ensemble.lasso.4096.trend"+".pkl")
    train_stat = signal_stat["train.stat"]
    good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
    train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    test_stat = signal_stat["test.stat"]
    test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))


# 600519 train sharpe  nan test sharpe  nan
# 000858 train sharpe  nan test sharpe  nan
# 000568 train sharpe  nan test sharpe  nan
# 600809 train sharpe  nan test sharpe  nan
# 002304 train sharpe  0.22738178582067017 test sharpe  -0.03630524094131039


600519 train sharpe  nan test sharpe  nan
000858 train sharpe  nan test sharpe  nan
000568 train sharpe  nan test sharpe  nan
600809 train sharpe  nan test sharpe  nan
002304 train sharpe  0.26985728009763355 test sharpe  0.09326742063043855


- it's not as good as thre model in week 9
- maybe we can use another way of ensemble
- we can generate 10 separate models with each pnl

# 2nd Ensemble

## Coefficients of all models

In [28]:
## generate 10 different models, and save their coefficients rather than calcualte their mean
nfold = 10
model_coef = dict([])
for i_mod in range(n_mod):
    train_std_mat = dict([])
    model_coef[str(i_mod)] = dict([])
    x_train = np.zeros((0, n_signal))
    y_train = np.array([])
    for product in product_list:
        cur_mat = all_train_array[product][i_mod,:,:]
        train_std_mat[product] = np.std(cur_mat, axis=0)
        x_train = np.vstack((x_train, cur_mat[:,:n_signal]/train_std_mat[product][:n_signal]))
        y_train = np.concatenate((y_train, cur_mat[:,n_signal]/train_std_mat[product][n_signal]))
    scaler =  StandardScaler(copy=True, with_mean=False, with_std=True)
    scaler.fit(x_train)
    x_std = np.sqrt(scaler.var_)
    x_train_normal = scaler.transform(x_train)
    model = LassoCV(n_alphas=100, fit_intercept=False, cv=10, max_iter=10000).fit(x_train_normal, y_train)
    fit_coef = model.coef_/x_std
    for product in product_list:
        coef = fit_coef*train_std_mat[product][n_signal]/train_std_mat[product][:n_signal]
        model_coef[str(i_mod)][product] = coef


In [29]:
save(model_coef, SAVE_PATH+"wine.ensemble.all.coef.pkl")

## Evaluate

In [30]:
for i_mod in range(n_mod):
    print(i_mod)
    strat = "wine.ensemble.lasso."+str(i_mod)+"."+str(period)
    for product in product_list:
        os.makedirs(SAVE_PATH+"/tmp pkl/"+product+"/"+strat, exist_ok=True)
        coef = model_coef[str(i_mod)][product]
        parLapply(CORE_NUM, all_dates, par_get_daily_pred, 
                  product=product, coef=coef, strat=strat, HEAD_PATH=SAVE_PATH, SAVE_PATH=SAVE_PATH)
        par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH=SAVE_PATH)    
        evaluate_signal("wine.ensemble.lasso."+str(i_mod), all_dates, product, 0.001, 20, CORE_NUM, SAVE_PATH, SAVE_PATH,
                            period=4096, split_str="2017",atr_filter=0.02, save_path="signal result atr", reverse=1)

0
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
1
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
2
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
3
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
4
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
5
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
6
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
7
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
8
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
9
reverse=1
reverse=1
reverse=1
reverse=1
reverse=1


In [31]:
min_pnl = 2
min_num = 10
for product in product_list:
    i = 0
    for i_mod in range(n_mod):
        strat = "wine.ensemble.lasso."+str(i_mod)+"."+str(period)
        signal_stat = load(SAVE_PATH+"/signal result atr/"+product+"."+strat+".trend.pkl")
        train_stat = signal_stat["train.stat"]
        good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
        if sum(good_strat)==0:
            continue
        train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        test_stat = signal_stat["test.stat"]
        test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        if i==0:
            test_all_pnl = np.zeros([len(test_pnl), n_mod])
            train_all_pnl = np.zeros([len(train_pnl), n_mod])
        test_all_pnl[:,i] = test_pnl
        train_all_pnl[:,i] = train_pnl
        i = i+1
    train_portfolio = np.array(np.mean(train_all_pnl[:,:i], axis=1))
    test_portfolio = np.array(np.mean(test_all_pnl[:,:i], axis=1))
    print(product, "train sharpe ", sharpe(train_portfolio), "test sharpe ", sharpe(test_portfolio))
    

# 600519 train sharpe  0.17117304776678074 test sharpe  0.2755301956905567
# 000858 train sharpe  0.46397312403353136 test sharpe  -1.0265335177621764
# 000568 train sharpe  nan test sharpe  nan
# 600809 train sharpe  nan test sharpe  nan
# 002304 train sharpe  0.2836953725557839 test sharpe  0.0834417960846937



600519 train sharpe  0.12703716704204124 test sharpe  -0.07853609641183407
000858 train sharpe  0.48411162192347634 test sharpe  -1.0232483096300733
000568 train sharpe  nan test sharpe  nan
600809 train sharpe  nan test sharpe  nan
002304 train sharpe  0.2804429717513608 test sharpe  0.05867656544930187


- we can see that the test result accpetable even though there is a small drawback
